In [1]:
# ! pip install tweepy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 1.0 MB/s eta 0:00:00


# Imports

In [1]:
import tweepy
import pandas as pd

# Load Bearer Token

In [4]:
import config_twitter

# from kaggle_secrets import UserSecretsClient
# user_secrets = UserSecretsClient()
# BEAER_TOKEN = user_secrets.get_secret("twitter-bearer-token")

BEARER_TOKEN = config_twitter.BEARER_TOKEN

# Client Set Up

In [6]:
client = tweepy.Client(bearer_token=BEARER_TOKEN)

In [7]:
elon = client.get_user(username="elonmusk")

# Get Tweets Demo

In [8]:
tweets = client.get_users_tweets(id='44196397',max_results=10,
                                 tweet_fields=["created_at,conversation_id"]).data

In [9]:
parsed_tweets = []
for tweet in tweets:
    parsed_tweets.append([tweet["created_at"],tweet['id'],tweet['text'],tweet['conversation_id']])

In [10]:
df = pd.DataFrame(parsed_tweets)

In [11]:
df.head()

,0,1,2,3
0,2022-10-25 23:26:39+00:00,1585050236120027137,Works on any moving land object https://t.co/X...,1585050236120027137
1,2022-10-25 13:54:37+00:00,1584906280082219009,@chazman @lexfridman Fasting rocks,1584764977457397760
2,2022-10-25 13:24:27+00:00,1584898685736869888,@VitalikButerin @balajis @zengjiajun_eth Had t...,1584625378253209603
3,2022-10-25 09:16:17+00:00,1584836233586348033,@dogeofficialceo @DirtyTesLa @rocobasilisk @Sw...,1584626500955144192
4,2022-10-25 08:30:54+00:00,1584824812345888768,@garyblack00 @SawyerMerritt One man’s hedge is...,1584640964609847296


# Get Tweets and Update CSV

In [12]:
parsed_tweets = []
parsed_tweets_only = []
for tweet in tweepy.Paginator(client.get_users_tweets, id='44196397',tweet_fields=["created_at,conversation_id"],
                                    max_results=100).flatten(limit=3200):
    parsed_tweets.append([tweet['id'],tweet["created_at"],tweet['text'],tweet['conversation_id']])
    if tweet['id'] == tweet['conversation_id']:
        parsed_tweets_only.append([tweet['id'],tweet["created_at"],tweet['text']])

In [13]:
tweets_and_replies_df = pd.DataFrame(parsed_tweets)
tweets_df = pd.DataFrame(parsed_tweets_only)

In [14]:
tweets_df.head()

,0,1,2
0,1585050236120027137,2022-10-25 23:26:39+00:00,Works on any moving land object https://t.co/X...
1,1584764977457397760,2022-10-25 04:33:08+00:00,What lesson took you the longest to unlearn?
2,1584717774261489665,2022-10-25 01:25:34+00:00,διαλεκτική
3,1584608031777902593,2022-10-24 18:09:30+00:00,RT @SpaceX: Starlink is now available in Jamai...
4,1584319362294775808,2022-10-23 23:02:25+00:00,RT @SpaceX: Falcon Heavy in the hangar at Laun...


In [15]:
tweets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 638 entries, 0 to 637
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype              
---  ------  --------------  -----              
 0   0       638 non-null    int64              
 1   1       638 non-null    datetime64[ns, UTC]
 2   2       638 non-null    object             
dtypes: datetime64[ns, UTC](1), int64(1), object(1)
memory usage: 15.1+ KB


### Get Last Saved Tweets

In [13]:
old_tweets_and_replies_df = pd.read_csv("../input/elon-musk-tweets-dataset/tweets_and_replies.csv", skiprows = 1, header = None)
old_tweets_df = pd.read_csv("../input/elon-musk-tweets-dataset/tweets.csv",skiprows = 1, header = None)

In [14]:
old_tweets_df.head()

,0,1,2
0,1577075264592351233,2022-10-03 23:16:58+00:00,RT @Tesla: The brain of your Tesla: Neural net...
1,1576994262226702336,2022-10-03 17:55:05+00:00,Let’s try this then: the will of the people wh...
2,1576976609348681738,2022-10-03 16:44:57+00:00,Please vote in this poll! ⬇️⬇️⬇️⬇️
3,1576969255031296000,2022-10-03 16:15:43+00:00,Ukraine-Russia Peace:\n\n- Redo elections of a...
4,1576840676284960769,2022-10-03 07:44:48+00:00,https://t.co/cGHSTdb9gv


### Merge Tweets

In [15]:
new_tweets = pd.concat([tweets_df,old_tweets_df]).drop_duplicates()
new_tweets_and_replies = pd.concat([tweets_and_replies_df,old_tweets_and_replies_df]).drop_duplicates()

In [16]:
new_tweets.to_csv("tweets.csv", header=["Id", "Date", "Text"], index=False)
new_tweets_and_replies.to_csv("tweets_and_replies.csv", header=["Id", "Date", "Text", "ConversationId"], index=False)

In [17]:
new_tweets.head()

,0,1,2
0,1577428272056389633,2022-10-04 22:39:41+00:00,"Buying Twitter is an accelerant to creating X,..."
1,1577075264592351233,2022-10-03 23:16:58+00:00,RT @Tesla: The brain of your Tesla: Neural net...
2,1576994262226702336,2022-10-03 17:55:05+00:00,Let’s try this then: the will of the people wh...
3,1576969255031296000,2022-10-03 16:15:43+00:00,Ukraine-Russia Peace:\n\n- Redo elections of a...
4,1576840676284960769,2022-10-03 07:44:48+00:00,https://t.co/cGHSTdb9gv
